# SQL Dump

Here i test `sqlite_utils` which is part of datasette ecosystem.

In [176]:
import requests
from copy import copy
from glom import glom, flatten
from dateutil.parser import parse as parse_dt
from sqlite_utils import Database

from pprint import pprint

In [3]:
BASE_URL = "https://launchlibrary.net/1.4.1/"

In [70]:
pads_raw = requests.get(BASE_URL + "pad/?limit=1000&mode=verbose").json()
locations_raw = requests.get(BASE_URL + "location/?limit=1000&mode=verbose").json()

In [71]:
agencies_raw = requests.get(BASE_URL + "agency?limit=1000&mode=verbose").json()
agency_types_raw = requests.get(BASE_URL + "agencytype?limit=1000&mode=verbose").json()

In [99]:
rockets_raw = requests.get(BASE_URL + "rocket?limit=1000&mode=verbose").json()
families_raw = requests.get(BASE_URL + "rocketfamily?limit=1000&mode=verbose").json()

In [144]:
launches_raw = requests.get(BASE_URL + "launch?limit=3000&mode=verbose").json()

In [361]:
missions_raw = requests.get(BASE_URL + "mission?limit=2000&mode=verbose").json()

JSONDecodeError: Unterminated string starting at: line 1 column 792528 (char 792527)

After some time I tried to refresh this request, but external api started to cut my requests and send incomplete JSONs

In [266]:
launch_statuses_raw = requests.get(BASE_URL + "launchstatus?limit=2000&mode=verbose").json()
agency_types_raw = requests.get(BASE_URL + "agencytype?limit=1000&mode=verbose").json()
mission_types_raw = requests.get(BASE_URL + "missiontype?limit=2000&mode=verbose").json()
payload_types_raw = requests.get(BASE_URL + "payloadtype?limit=2000&mode=verbose").json()

payload_types_raw[0]

{'types': [{'id': 0,
   'name': 'Satellite',
   'description': '',
   'changed': '2017-02-21 00:00:00'},
  {'id': 1,
   'name': 'Cubesat',
   'description': '',
   'changed': '2017-02-21 00:00:00'},
  {'id': 2,
   'name': 'Supplies',
   'description': '',
   'changed': '2017-02-21 00:00:00'},
  {'id': 3,
   'name': 'Crew',
   'description': '',
   'changed': '2017-02-21 00:00:00'},
  {'id': 4,
   'name': 'Probe',
   'description': '',
   'changed': '2017-02-21 00:00:00'},
  {'id': 5,
   'name': 'Rover',
   'description': '',
   'changed': '2017-02-21 00:00:00'},
  {'id': 6,
   'name': 'Lander',
   'description': '',
   'changed': '2017-02-21 00:00:00'},
  {'id': 7,
   'name': 'Habitat',
   'description': '',
   'changed': '2017-02-21 00:00:00'},
  {'id': 8,
   'name': 'Telescope',
   'description': '',
   'changed': '2017-02-21 00:00:00'},
  {'id': 9,
   'name': 'Other',
   'description': '',
   'changed': '2017-02-21 00:00:00'}],
 'total': 10,
 'count': 10,
 'offset': 0}

In [ ]:
for raw in []

In [348]:
db = Database("launchlibrary.db")

pads = db["pads"]
locations = db["locations"]

agencies = db["agencies"]

rockets = db["rockets"]
families = db["rocketfamilies"]

missions = db["missions"]
payloads = db["payloads"]

launches = db["launches"]

In [349]:
agency_types = {t["id"]: t["name"].upper() for t in agency_types_raw["types"]}
status_types = {t["id"]: t["name"].upper() for t in launch_statuses_raw["types"]}
mission_types = {t["id"]: t["name"].upper() for t in mission_types_raw["types"]}
payload_types = {t["id"]: t["name"].upper() for t in payload_types_raw["types"]}
payload_types

{0: 'SATELLITE',
 1: 'CUBESAT',
 2: 'SUPPLIES',
 3: 'CREW',
 4: 'PROBE',
 5: 'ROVER',
 6: 'LANDER',
 7: 'HABITAT',
 8: 'TELESCOPE',
 9: 'OTHER'}

In [350]:
def ids_to_m2m(cur, ids, lookup, m2m_table):
    for _id in ids:
        cur.m2m(lookup.name, lookup={"id": _id}, m2m_table=m2m_table)

In [351]:
print(agencies_raw["agencies"][0].keys())
_agencies = glom(agencies_raw["agencies"],
    [{
        "id":  "id",
        "name":"name",
        "countryCode": "countryCode",
        'abbrev': "abbrev", 
        'type': ("type", lambda t: agency_types[t]),
        'islsp': "islsp", 
#         'changed': ("changed", parse_dt),
        'infoURL': "infoURL", 
        'wikiURL': "wikiURL", 
        
    }]
)
agencies.upsert_all(_agencies, pk="id")

dict_keys(['id', 'name', 'countryCode', 'abbrev', 'type', 'infoURL', 'wikiURL', 'infoURLs', 'islsp', 'changed'])


<Table agencies (id, name, countryCode, abbrev, type, islsp, infoURL, wikiURL)>

In [352]:
print(locations_raw["locations"][0].keys())
_locations = glom(locations_raw["locations"],
    [{
        "id":  "id",
        "name":"name",
#         'changed': ("changed", parse_dt),
        'infoURL': "infoURL", 
        'wikiURL': "wikiURL", 
        
    }]
)

locations.upsert_all(_locations, pk="id")

dict_keys(['id', 'name', 'countrycode', 'infoURL', 'wikiURL', 'infoURLs', 'changed'])


<Table locations (id, name, infoURL, wikiURL)>

In [ ]:
print(pads_raw["pads"][0].keys())
# droping agencies (its many to many relation), anf info URLs
_pads = copy(pads_raw["pads"])
_pads = glom(_pads,
    [{
        "id":  "id",
        "name":"name",
        "padType": "padType",
        'latitude': "latitude", 
        'longitude': "longitude", 
        'retired': "retired", 
        'locationId': "locationid", 
        'agencies': 'agencies',
#         'changed': ("changed", parse_dt),
        'infoURL': "infoURL", 
        'wikiURL': "wikiURL", 
        
    }]
)
# pads.upsert_all(_pads, pk="id",
#     foreign_keys=[
#     ("locationId", locations.name, "id")
# ])

for r in _pads:
    a_ids = glom(r.pop("agencies"), ["id"], default=[])
    cur = pads.upsert(r, pk="id", foreign_keys=[("locationId", locations.name, "id")])
    if a_ids is not None:
        ids_to_m2m(cur, a_ids, agencies, "pad_agency")

In [ ]:
print(families_raw["RocketFamilies"][0].keys())
_families = copy(families_raw["RocketFamilies"])
_families = glom(_families,
    [{
        "id":  "id",
        "name":"name",
        "agencies": "agencies", #TODO: m2m
#         'changed': ("changed", parse_dt),
        
    }]
)
# _families[0]
# families.upsert_all(_families, pk="id",
# )
for r in _families:
    a_ids = glom(r.pop("agencies"), ["id"], default=[])
    cur = families.upsert(r, pk="id")
    if a_ids is not None:
        ids_to_m2m(cur, a_ids, agencies, "rocketfamily_agency")

In [355]:
print(rockets_raw["rockets"][0].keys())
_rockets = glom(rockets_raw["rockets"],
    [{
        "id":  "id",
        "name":"name",
        "configuration": "configuration",
        'defaultPads': "defaultPads", # TODO m2m
        'familyId': "family.id", 
#         'changed': ("changed", parse_dt),
        'infoURL': "infoURL", 
        'wikiURL': "wikiURL", 
        'imageURL': "imageURL",
    }]
)
print(_rockets[0])
for r in _rockets:
    default_pads = r.pop("defaultPads")
    
    cur = rockets.upsert(r, pk="id",
        foreign_keys=[
        ("familyId", families.name, "id")
    ])
    
    # add m2m relation between pads and rockets
    if default_pads is not None:
        default_pads = (int(p) for p in filter(None, set(default_pads.replace(" ", "").split(","))))
        ids_to_m2m(cur, default_pads, pads, "rocket_defaultpad")



dict_keys(['id', 'name', 'configuration', 'defaultPads', 'family', 'infoURL', 'wikiURL', 'infoURLs', 'imageSizes', 'imageURL', 'changed'])
{'id': 1, 'name': 'Falcon 9 v1.1', 'configuration': '9 v1.1', 'defaultPads': '84,100', 'familyId': 13, 'infoURL': None, 'wikiURL': 'http://en.wikipedia.org/wiki/Falcon_9', 'imageURL': 'https://launchlibrary1.nyc3.digitaloceanspaces.com/RocketImages/Falcon9v1.1.jpg_1280.jpg'}


In [356]:
_launches = launches_raw["launches"]
print(_launches[0].keys()) # TODO add rest of the keys
_launches = glom(_launches,
    [{
        "id":  "id",
        "name":"name",
        'rocketId': "rocket.id", 
        'lspId': "lsp.id",
        "padId": "location.pads.0.id",
        "status": ("status", lambda t: status_types[t]),
        "windowstart": ("isostart", parse_dt),
        'windowend': ("isoend", parse_dt), 
        "net": ("isonet", parse_dt),
#         'changed': ("changed", parse_dt),
        'infoURL': "infoURL", 
    }]
)
pprint(_launches[0])
launches.upsert_all(_launches, pk="id",
    foreign_keys=[
        ("rocketId", rockets.name, "id"),
        ("lspId", agencies.name, "id"),
        ("padId", pads.name, "id")
])

dict_keys(['id', 'name', 'windowstart', 'windowend', 'net', 'wsstamp', 'westamp', 'netstamp', 'isostart', 'isoend', 'isonet', 'status', 'inhold', 'tbdtime', 'vidURLs', 'vidURL', 'infoURLs', 'infoURL', 'holdreason', 'failreason', 'tbddate', 'probability', 'hashtag', 'changed', 'location', 'rocket', 'missions', 'lsp'])
{'id': 1829,
 'infoURL': None,
 'lspId': 161,
 'name': 'Atlas LV-3A | Samos 2',
 'net': datetime.datetime(1961, 1, 31, 20, 21, 19, tzinfo=tzutc()),
 'padId': 99,
 'rocketId': 193,
 'status': 'SUCCESS',
 'windowend': datetime.datetime(1961, 1, 31, 20, 21, 19, tzinfo=tzutc()),
 'windowstart': datetime.datetime(1961, 1, 31, 20, 21, 19, tzinfo=tzutc())}


<Table launches (id, name, rocketId, lspId, padId, status, windowstart, windowend, net, infoURL)>

In [ ]:
print(missions_raw["missions"][0].keys())

_missions = glom(missions_raw["missions"],
    [{
        "id":  "id",
        "name":"name",
        "description": "description",
        "agencies": "agencies",
#         'changed': ("changed", parse_dt),
        "type": ("type", lambda t: mission_types.get(t, None)),
        'infoURL': "infoURL", 
        'wikiURL': "wikiURL", 
    }],
)
pprint(_missions[0])
# pprint(glom(missions_raw["missions"], ["type"]))

for r in _missions:
    a_ids = glom(r.pop("agencies"), ["id"], default=[])
    cur = missions.upsert(r, pk="id")
    if a_ids is not None:
        ids_to_m2m(cur, a_ids, agencies, "mission_agency")

In [ ]:
# this won't work because some dims are 2D
def split_dims(p):
    if p["dimensions"] is not None:
        x, y, z = (float(d) for d in p["dimensions"].split("x"))
    else:
        x = y = z = None
    del p["dimensions"]
    p["dimX"] = x
    p["dimY"] = y
    p["dimZ"] = z
    

_payloads = flatten(glom(missions_raw["missions"],
    ["payloads"],
))
print(_payloads[0].keys())

payload_mission_id = {}
for m in (glom(missions_raw["missions"], [{"missionId":"id", "payloads": "payloads"}],)):
    for p in m["payloads"]:
        payload_mission_id[p["id"]] = m["missionId"]
    
_payloads = glom(_payloads,
    [{
        "id": "id",
        "name": "name",
        'countryCodes': 'countryCodes',
        'weight': ('weight', lambda f: float(f) if f is not None else f),
        'dimensions': "dimensions",
        "agencies": "agencies",
        'description': 'description',
        "total": "total",
        "type": ("type", lambda t: payload_types.get(t, None)), # TODO: WTF is this
        "missionId": ("id", lambda i: payload_mission_id[i]),
    }]
)
# _payloads = [split_dims(p) for p in _payloads]
# payloads.upsert_all(_payloads, pk="id",
#     foreign_keys=[
#         ("missionId", missions.name, "id"),
# ])

for r in _payloads:
    a_ids = glom(r.pop("agencies"), ["id"], default=[])
    cur = payloads.upsert(r, pk="id", foreign_keys=[("missionId", missions.name, "id"),])
    if a_ids is not None:
        ids_to_m2m(cur, a_ids, agencies, "payload_agency")

In [ ]:
launches_raw["launches"][1000]